In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import os
import pandas as pd
import numpy as np
from astropy import units as u

In [2]:
def count_fields(df, w):
    grp = df[w].groupby('fieldID')
    nobs = grp['expMJD'].agg(len)
    nobs.name = 'nobs'
    return nobs

In [3]:
# read in field corners
fields = pd.read_csv('../data/ztf_field_corners.csv.gz',index_col='field')

In [4]:
# Convert field corners to healpix pixels
import healpy as hp
nside = 2048
n_pix = hp.nside2npix(nside)
field_pixels = {}
for idx, field in fields.iterrows():   
    ra = np.array([field['ra1'], field['ra2'], field['ra3'], field['ra4']])
    dec = np.array([field['dec1'], field['dec2'], field['dec3'], field['dec4']])

    theta = np.radians(90.0 - dec)
    phi = np.radians(ra)

    vertices = hp.ang2vec(theta, phi)
    list_of_pixels = hp.query_polygon(nside, vertices)
    field_pixels[idx] = list_of_pixels


In [5]:
def fill_pixels(df):
    m = np.zeros(n_pix)
    for field, nobs in df.iteritems():
        m[field_pixels[field]] += nobs
    #m[m < 1] = hp.UNSEEN
    return m 

In [6]:
import ligo.skymap.plot.allsky as pas
from matplotlib.projections import projection_registry
from astropy.visualization.wcsaxes.frame import EllipticalFrame

In [7]:
class AntiCenterAxes(pas.AutoScaledWCSAxes):
    """Base class for a multi-purpose all-sky projection"""

    def __init__(self, *args, **kwargs):
        kwargs = dict(kwargs)
        obstime = kwargs.pop('obstime', None)
        header = {
            'NAXIS': 2,
            'NAXIS1': 360,
            'NAXIS2': 180,
            'CRPIX1': 180.5,
            'CRPIX2': 90.5,
            'CRVAL1': 0.0,
            'CRVAL2': 0.0,
            'CDELT1': -2 * np.sqrt(2) / np.pi,
            'CDELT2': 2 * np.sqrt(2) / np.pi,
            'CTYPE1': self._xcoord + '-' + self._wcsprj,
            'CTYPE2': self._ycoord + '-' + self._wcsprj,
            'RADESYS': self._radesys}
        if obstime is not None:
            header['DATE-OBS'] = Time(obstime).utc.isot
        super().__init__(
            header, *args, frame_class=EllipticalFrame, **kwargs)
        self.coords[0].set_ticks(spacing=30 * u.deg)
        self.coords[1].set_ticks(spacing=30 * u.deg)
        self.coords[0].set_ticklabel(exclude_overlapping=True)
        self.coords[1].set_ticklabel(exclude_overlapping=True)
        #self.coords[1].set_ticklabel_position('lbt')

In [8]:
class AstroDegreesAitoffAntiCenterAxes(pas.Astro, pas.Degrees, pas.Aitoff, AntiCenterAxes):
    name = 'astro degrees aitoff anticenter'

In [9]:
projection_registry.register(AstroDegreesAitoffAntiCenterAxes)        


In [10]:
cmaps = {'g':'Blues','r':'Oranges','i':'RdPu'}
vmax = {(3,'g'):200,(3,'r'):200,(3,'i'):50}


In [11]:
from sqlalchemy import create_engine

path = '/Users/annaho/Dropbox/Projects/Research/Koala/docs/anna.db'
engine = create_engine('sqlite:///%s' %path)

df = pd.read_sql('Summary', engine)

In [12]:
prop = 3
for filt in ['g','r']:
    nobs = count_fields(df, (df['propID'] == prop) & (df['filter'] == filt) & 
                        ~df['subprogram'].str.startswith('reference') & ~df['subprogram'].str.startswith('ToO'))
    m = fill_pixels(nobs)
        
    cmap = plt.cm.get_cmap(cmaps[filt])
    cmap.set_under("white")


    fig = plt.figure()
    ax = plt.axes(
        [0.05, 0.05, 0.9, 0.9],
        projection='astro degrees aitoff anticenter')
    mappable = ax.imshow_hpx(m,cmap=cmap, vmin=1,vmax=vmax[(prop,filt)]) 
    plt.xlabel('RA')
    plt.ylabel('Dec')

    plt.colorbar(mappable=mappable,orientation='horizontal',shrink=0.5,label='Number of epochs')
    plt.savefig('coverage_%s_%s.png' %(prop, filt),bbox_inches='tight',dpi=300)

/Users/annaho/anaconda/lib/python3.5/site-packages/astropy_healpix/core.py:519: RuntimeWarning: invalid value encountered in bilinear_interpolation_weights
  result = _core.bilinear_interpolation_weights(lon, lat, nside)
/Users/annaho/anaconda/lib/python3.5/site-packages/astropy_healpix/core.py:519: RuntimeWarning: invalid value encountered in bilinear_interpolation_weights
  result = _core.bilinear_interpolation_weights(lon, lat, nside)
